### Examples Of Pyspark ML

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

import pyspark.pandas as ps

/home/rakesh/Downloads/Projects/Pyspark-With-Python/venv/lib/python3.12/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
spark = SparkSession.builder.appName('Missing').getOrCreate()

25/01/03 13:07:43 WARN Utils: Your hostname, rakesh resolves to a loopback address: 127.0.1.1; using 192.168.1.103 instead (on interface wlp4s0)
25/01/03 13:07:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/03 13:07:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/03 13:07:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
## Read The dataset
training = spark.read.csv('test1.csv',header=True,inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [6]:
training.columns[1:-1]

['age', 'Experience']

In [7]:
## Pyspark.pandas - pandas like api, but works on spark cluster
training = ps.read_csv('test1.csv')
training

training.info()

training.columns

training.columns[1:-1]

training = training.to_spark()
training.show()

/home/rakesh/Downloads/Projects/Pyspark-With-Python/venv/lib/python3.12/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        6 non-null      object
 1   age         6 non-null      int32 
 2   Experience  6 non-null      int32 
 3   Salary      6 non-null      int32 
dtypes: int32(3), object(1)+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



/home/rakesh/Downloads/Projects/Pyspark-With-Python/venv/lib/python3.12/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [8]:
## [Age,Experience]----> new feature--->independent feature
# featureassembler = VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")
featureassembler = VectorAssembler(inputCols=training.columns[1:-1], outputCol="Independent Features")
output = featureassembler.transform(training)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [9]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [10]:
finalized_data = output.select(["Independent Features","Salary"])
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [11]:
##train test split
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

25/01/03 13:07:55 WARN Instrumentation: [ee1f0e16] regParam is zero, which might cause numerical instability and overfitting.
25/01/03 13:07:55 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [12]:
### Coefficients
regressor.coefficients

DenseVector([-90.9091, 1272.7273])

In [13]:
### Intercepts
regressor.intercept

17545.454545455385

In [14]:
### Prediction
pred_results = regressor.evaluate(test_data)

In [15]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|16909.090909091006|
|          [24.0,3.0]| 20000|19181.818181818242|
|         [31.0,10.0]| 30000|27454.545454545525|
+--------------------+------+------------------+



In [16]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

25/01/03 13:07:59 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


(1757.5757575757464, 3597796.1432506596)